## Импорт и определение библиотек

In [1]:
# отключение параллелизма
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
# ставим разные пакеты оттуда
!pip install langchain langchain-community langchain_huggingface unstructured pypandoc -q
!pip install faiss-cpu -q
!pip install langchain-ollama -q

In [2]:
# импортируем библиотеки
from langchain.document_loaders import WebBaseLoader, PyPDFLoader, UnstructuredEPubLoader
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
from langchain.tools import Tool
from langchain.agents import initialize_agent
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings
import tqdm as notebook_tqdm
from langchain_community.vectorstores import FAISS
from langchain.llms import Ollama
from langchain_ollama import OllamaLLM
from langchain.text_splitter import TextSplitter
from typing import List

USER_AGENT environment variable not set, consider setting it to identify your requests.


# Готовим коктейль с помощью RAG
## Чутка `embeddings` с `HuggingFace` для оснастки

In [4]:
# Чтение токена из файла
def read_token(file_path): 
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith("HUGGINGFACEHUB_API_TOKEN"):
                return line.split('=')[1].strip()
    raise ValueError("Token not found in the specified file.")

# Загрузка токена
hf_token = read_token('../.env')

# Формирование эмбеддингов
hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru",  
    #"sentence-transformers/paraphrase-multilingual-mpnet-base-v2", 
    model_kwargs={"device": "cpu", "token": hf_token},
    encode_kwargs={
        'normalize_embeddings': True  # Для косинусной схожести
    }
)

/home/elijah/Documents/LLM/cocktail_qr/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Можно пропустить (если используете наполненную чашу векторного хранилища)
### Немного `Document Loader`

In [10]:
# Загрузка EPUB файла от бармена-литературоведа
loader = UnstructuredEPubLoader('../docs/Federle.epub', show_progress=True)
doc_epub_mistral = loader.load()

# смотрим, скока получилось загрузить
print('pages count:', len(doc_epub_mistral))
print('max chars on page:', max([len(page.page_content) for page in doc_epub_mistral]))

pages count: 1
max chars on page: 107261


In [ ]:
"""# Загружаем книгу от крутого бармена
#loader = WebBaseLoader(url)
loader = PyPDFLoader('../docs/Bortnik_1000.pdf')
data_pdf = loader.load()

print('pages count:', len(data_pdf))
print('max chars on page:', max([len(page.page_content) for page in data_pdf]))"""

# на тот случай, если решим загружать подобную литературу пачкой pdf
"""
#!pip install unstructured "unstructured[pdf]" -q
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader("../docs", glob="**/*.pdf", show_progress=True)
data = loader.load()"""

### Исполним `Text splitters` и разобьём на чанки

In [11]:
class RecipeSplitter(TextSplitter):
    def __init__(self):
        super().__init__(chunk_size=1000, chunk_overlap=300)
    
    def split_text(self, text: str) -> List[str]:
        # Убираем все пустые строки и лишние пробелы
        cleaned_text = "\n".join([line.strip() for line in text.split("\n") if line.strip()])
        
        # Разделяем текст на рецепты по маркеру "НАЗВАНИЕ_РЕЦЕПТА\nАВТОР"
        chunks = []
        current_chunk = []
        lines = cleaned_text.split("\n")
        
        i = 0
        while i < len(lines):
            line = lines[i]
            # Проверяем начало нового рецепта
            if i + 1 < len(lines) and lines[i+1].isupper() and "(" in lines[i+1] and ")" in lines[i+1]:
                if current_chunk:
                    chunks.append("\n".join(current_chunk))
                    current_chunk = []
                current_chunk.append(line)
                current_chunk.append(lines[i+1])
                i += 2
                continue
            
            current_chunk.append(line)
            i += 1
        
        if current_chunk:
            chunks.append("\n".join(current_chunk))
        
        return chunks
    
splitter = RecipeSplitter()

In [12]:
chunks = splitter.split_documents(doc_epub_mistral)
print('Количество чанков:', len(chunks))
print('Максимальное количество символов в чанке:', max([len(chunk.page_content) for chunk in chunks]))

Количество чанков: 73
Максимальное количество символов в чанке: 15125


### Наполним чашу `Vector Store`

In [17]:
# Если нужно сохранить новый документ в базу
try:
    db_embed = FAISS.from_documents(
            chunks, 
            hf_embeddings_model
        )
except Exception as e:
    raise RuntimeError(f"Ошибка при создании базы данных FAISS: {e}")


# Сохраняем векторную базу локально
db_embed.save_local("../data/faiss_db_epub_mistral")

## Нужно запустить (если используете готовое векторное хранилище)
### Испьём чашу `Vector Store`

In [18]:
# Если нужно изъять из уже сохранённого документа из базы
# Загрузка векторной базы данных из локального файла
db_embed = FAISS.load_local(
        "../data/faiss_db_epub_mistral", 
        hf_embeddings_model,
        allow_dangerous_deserialization = True
    )

In [ ]:
# альтернативный подход для гибридного поиска
# возможно нужно будет позже вернуться
"""!pip install rank_bm25 -q

from langchain.retrievers import BM25Retriever, EnsembleRetriever

# Создаем гибридный поиск (семантический + ключевые слова)
bm25_retriever = BM25Retriever.from_documents(split_documents)
bm25_retriever.k = 3

faiss_retriever = db_embed.as_retriever(search_kwargs={"k": 1})

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.2, 0.8]
)

# Использование:
processed_query = "ингредиенты: " + query  # Добавляем контекст для поиска
results = ensemble_retriever.invoke(processed_query)
for result in results:
    print(result.metadata['ingredients'])"""

In [30]:
query = "джин, лимонный сок, черносмородиновый ликер"
results = db_embed.similarity_search(query, k=2)
for result in results:
    print(result.page_content)
    print()

Лорд Крюшон
ЛОРД ДЖИМ (1899)
ДЖОЗЕФ КОНРАД
Помните мистера Марлоу из «Сердца тьмы»? (Да бросьте, это было всего-то десять рецептов назад.) На сей раз наш знакомец ведет рассказ не о себе. Молодой моряк Джим грезил героическими подвигами – и бросил на произвол судьбы терпящее бедствие судно с направлявшимися в Мекку паломниками. (На заметку: для быстрого обогащения на литературной ниве возвращайтесь в позапрошлый век и описывайте человеческие драмы на фоне морской пучины.) Хотя повествование ведется от лица нескольких рассказчиков, а нарушенная хронология событий должна бы добавлять интриги, «Лорд Джим» местами тяжеловат. Предлагаем оживить постных британцев их же знаменитым крюшоном (этот напиток любого тихоню превратит в героя):
1 огурец, нарезанный кружочками, и ломтик для украшения;
60 мл крюшона «Пиммз № 1» (Pimm’s № 1);
1 банка (0,33 л) лимонно-лаймовой газировки (кстати, «Буратино» или «Лимонад» отлично подойдут для подобных миксов!);
долька лимона.
Помещаем в коллинз несколько к

# Месье системный промт

In [ ]:
system_prompt = """
        Ты — помощник для поиска рецептов коктейлей. У тебя есть доступ к базе данных с рецептами, вдохновлёнными литературными произведениями.
        Когда пользователь вводит ингредиенты, используй инструмент "Cocktail Recipe Finder", чтобы найти подходящие рецепты.
        Если рецепт не найден, предложи альтернативные варианты или уточни запрос.

        ===
        
        Пример:
        Пользователь: Какие коктейли можно сделать из водки и апельсинового сока?
        Агент: Использую инструмент "Cocktail Recipe Finder" для поиска рецептов. Вот что найдено: [рецепт].

        ===

        Пример корректного ответа по введённым ингредиентам (ржаной виски, грейпфрутовый сок):
        Коктейль "Рожь и предубеждение"
        Состав:
        - Ржаной виски (50 мл)
        - Грейпфрутовый сок (90 мл)
        Способ приготовления:
        - Ингредиенты выливаем в стакан рокс поверх кубиков льда, размешиваем в ритме «сердце вскачь». 
        - Хотим подчеркнуть, юные леди, мы нисколечко не предубеждены против замужества, а лишь призываем вас уяснить главное: чтобы почувствовать себя королевой, не нужно и дворца (как, впрочем, и короля).
    """

In [ ]:
# Промпт для LangChain
qa_prompt = PromptTemplate(
    template=system_prompt + "\nКонтекст: {context}\nВопрос: {question}\nОтвет:",
    input_variables=["style", "context", "question"]
)

## Полирнём `Retriver` и инициализирует `llm` для вкуса

In [ ]:
# Инициализация модели
llm = OllamaLLM(
    model="mistral:instruct",
    #temperature=0.2,          # Для креативности
    num_predict=512,         # Увеличенная длина ответа
    stop=["\\n\\n", "###"],   # Стоп-токены
    #top_k=40
)

In [ ]:
# Модифицируем параметры ретривера
retriever = db_embed.as_retriever(
    search_type="mmr",              # Maximal Marginal Relevance
    search_kwargs={
        "k": 3,                     # Уменьшаем количество возвращаемых документов
        "fetch_k": 20,              # Уменьшаем общий пул для поиска
        #"lambda_mult": 0.5          # Баланс между релевантностью и разнообразием
    }
)

# Инициализация цепочки RetrievalQA
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# Миксуем рецепт коктейля `agent` 00х

In [ ]:
# Создание инструмента
agent_tools = [
    Tool(
        name="Cocktail Recipe Finder",
        func=retrieval_qa.run,
        description="Используй этот инструмент, чтобы найти рецепты коктейлей по заданным ингредиентам. Вводи список ингредиентов, и инструмент вернёт подходящие рецепты. Пример: ['сок', 'виски']."
    )
]

# создание агента
agent = initialize_agent(
    tools=agent_tools, 
    llm=llm, 
    agent="conversational-react-description",       #"zero-shot-react-description", "plan-and-execute"
    verbose=True,
    system_prompt=system_prompt                     # не забыть указать системный промт для вкуса
)

# Задаём общие параметры, после чего коктейль готов
## Вначале коктейль

In [ ]:
# Объявление функции на ввод данных от пользователя
def get_user_input():
    """
    Запрашивает у пользователя ингредиенты для поиска рецептов.
    """
    user_input = input("Введите ингредиенты, разделенные запятыми: ")
    ingredients = [ingredient.strip() for ingredient in user_input.split(",")]
    return ingredients


# Обработка ответа
def format_response(response):
    if "не знаю" in response.lower() or "не найдено" in response.lower():
        return "К сожалению, я не нашёл коктейлей с этими ингредиентами. Попробуйте другие ингредиенты."
    else:
        return response
    

# Функция выбора стиля ответа
def choose_style(styles):
    chosen_style = input("Выберите номер стиля (по умолчанию 4): ") or "4"
    return styles.get(chosen_style, styles["4"])#["name"]

In [ ]:
def main():
    # Определение стилей
    styles = {
        "1": "стиль космического ужаса и хтонического мрака Говарда Ф. Лавкрафта",
        "2": "гопническо-быдляцкий жаргон",
        "3": "экспериментальный стиль нарезок Уильяма Берроуза",
        "4": "стиль без изменений"  # Стиль по умолчанию
    }
    
    while True:
        ingredients = get_user_input()
        if not ingredients:
            print("Вы не ввели ингредиенты. Попробуйте снова.")
            continue
        
        # Преобразуем массив ингредиентов в строку
        ingredients_str = ", ".join(ingredients)
        
        # Формируем текстовый запрос
        user_prompt = (
            """Напиши, какие коктейли можно изготовить из представленных ниже ингредиентов. 
            Для каждого коктейля опиши подробный способ приготовления и укажи пропорции. 
            Отвечай строго на русском языке, используя чёткий и лаконичный формат. 
            Не добавляй лишнюю информацию, которая не относится к рецепту. 
            Ингредиенты: {ingredients}""".format(ingredients=ingredients_str)
        )
        response = retrieval_qa.invoke({"query": user_prompt})  
        
        # Получаем ответ без вывода его на экран
        original_response = response["result"]
        # Выбор стиля
        style = choose_style(styles)
        prompt = PromptTemplate(input_variables=['output_text', 'style'],
                                template='''Добро пожаловать в поиск коктейлей!\nПерепиши этот текст в заданном стиле: {output_text}\nСтиль: {style}.\nРезультат:\n''')
        
        # Генерация текста с использованием оригинального ответа и выбранного стиля
        styled_prompt = prompt.format(output_text=original_response, style=style)
        
        # Применение выбранного стиля к оригинальному ответу
        styled_response = llm.invoke(styled_prompt)
        print(styled_response)
        break
    
    return styled_response

if __name__ == "__main__":
    # Сохраняем результат в переменную
    recipe = main()